In [1]:
import numpy as np
import string
import itertools
import time
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/amit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_wiki_data():
    wiki_file = open('/Users/amit/Box Sync/lsda-2018/wikipedia/wiki-text.txt', 'r').readlines()[0]
    print(len(wiki_file))
    num_chunks = 1600
    chunks, chunk_size = len(wiki_file), np.int32(len(wiki_file)/num_chunks)
    wiki_chunks = [wiki_file[i:i+chunk_size] for i in np.int32(np.arange(0, chunks, chunk_size))]
    return wiki_chunks
    
    
wiki_data = load_wiki_data()
print(len(wiki_data))

713069767
1601


In [3]:
t1=time.time()
all_words=[]

for w in wiki_data:
    all_words.append(w.split())
aw=[w for a in all_words for w in a if not w in stop_words]
print(len(aw))
print("Time to split",time.time()-t1)

81535178
Time to split 20.645808219909668


In [68]:
t1=time.time()
all_words=aw
s_all_words = sorted(all_words)
print(len(s_all_words))
gg=itertools.groupby(s_all_words)
word_counts=[(k,len(list(g))) for k,g in gg]

dvocab={}
t=0
for i,x in enumerate(word_counts):
    if (x[1]>500):
        dvocab[x[0]]=t
        t+=1

clean_words=[]
for w in all_words:
    if w in dvocab:
        clean_words.append((w,dvocab[w]))
print("Time to create vocab and clean words",time.time()-t1)
print('Number of clean words',len(clean_words))
print('Vocab size',len(list(dvocab.keys())))

81535178
Time to create vocab and clean words 102.00454592704773
Number of clean words 71618326
Vocab size 13201


In [10]:
def cooccurence(words, wnd_size):
    rv = {}
    for kk in np.arange(1, len(words),1):
        w_ii = words[kk][0]
        ind_ii=words[kk][1]
        left = max(0, kk - wnd_size)
        for jj in np.arange(left,kk,1):
            w_jj=words[jj][0]
            ind_jj=words[jj][1]
            if (w_ii, w_jj) not in rv:
                rv[(w_ii, w_jj)] = [1.,ind_ii,ind_jj]
            else:
                rv[(w_ii, w_jj)][0] += 1.
    return(rv)

In [70]:
def cooccurence_full(words, wnd_size):
    MM=np.zeros((vocab_size,vocab_size),dtype=np.int32)
    for kk in np.arange(1, len(words),1):
        w_ii = words[kk][0]
        ind_ii=words[kk][1]
        left = max(0, kk - wnd_size)
        for jj in np.arange(left,kk,1):
            w_jj=words[jj][0]
            ind_jj=words[jj][1]
            MM[ind_ii,ind_jj]+=1
           
    return(MM)

In [71]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import numpy as np

print('len clean_words',len(clean_words))
vocab_size = len(list(dvocab.keys()))
print(vocab_size)

t1=time.time()
D=cooccurence_full(clean_words,5)
print("Time to compute matrix",time.time()-t1)

D=np.float32(D)
t1=time.time()
M = D + D.T
word_cnt_vec = D.sum(axis=1).T
M = np.count_nonzero(M) * (M + 1)
M = np.log(M / word_cnt_vec.dot(word_cnt_vec.T))
U,s,V = svds(csr_matrix(M), k=50)
W = U.dot(np.diag(np.sqrt(s)))
print("Time to compute matrix and svd",time.time()-t1)

len clean_words 71618326
13201
Time to compute matrix 832.2892348766327
Time to compute matrix and svd 82.22771096229553


In [66]:
i=np.mod(np.argmax(D),3078)
j=np.argmax(D)//3078
print(i,j)
print(list(dvocab.keys())[1911])

1969 1911
nine


In [72]:
word_to_vec={}

for i,w in zip(dvocab.values(),dvocab.keys()):
    word_to_vec[w]=W[i,:]

In [73]:
def find_closest_word(word,K):
    words = list(word_to_vec.keys())
    target = word_to_vec[word]
    dist = np.array([np.linalg.norm(target-word_to_vec[el]) for el in words])
    ind = np.argsort(dist)[1:(K+1)]
    print(ind)
    rv = [words[ii] for ii in ind]
    return rv

In [78]:
def find_closest_word_to_v(v,K):
    words = list(word_to_vec.keys())
    target = v
    dist = np.array([np.linalg.norm(target-word_to_vec[el]) for el in words])
    ind = np.argsort(dist)[1:(K+1)]
    print(ind)
    rv = [words[ii] for ii in ind]
    return rv

In [85]:
v=word_to_vec['human']-word_to_vec['baby']+word_to_vec['cow']
find_closest_word_to_v(v,5)

[7877 5726 8786  507 6572]


['nature', 'humans', 'physical', 'animal', 'knowledge']